In [20]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json

#create new connection to test instance, given an alias 'flagonTest' for later reference
#note: connections are easy enough such that examples are enough, no need for further abstraction
flagonClient = connections.create_connection('flagonTest', hosts=['localhost'], timeout=60)

#TODO describe connections

#hello world test
print(flagonClient)

<Elasticsearch([{'host': 'localhost'}])>


In [4]:
#create new search object, using cluster alias and index name 
#TODO abstract to Query Builder
AleS = Search(using='flagonTest', index="userale")

#TODO MetaData on Hits
#print Hello World
for hit in AleS.scan():
    print(hit.to_dict())


{'path': ['a.ui big red button', 'div.ui center aligned text container', 'div.ui very padded vertical segment', 'div.main-wrapper', 'body', 'html', '#document', 'Window'], 'userId': None, 'useraleVersion': 'demo', 'location': {'x': 462, 'y': 688}, 'toolName': None, 'toolVersion': None, 'clientTime': 1569123638986, 'details': {'alt': False, 'shift': False, 'meta': False, 'clicks': 1, 'ctrl': False}, '@timestamp': '2019-09-22T03:40:39.612Z', 'tags': ['_grokparsefailure'], 'type': 'mouseup', 'host': '172.18.0.1', 'target': 'a.ui big red button', 'userAction': True}
{'path': ['a.ui big red button', 'div.ui center aligned text container', 'div.ui very padded vertical segment', 'div.main-wrapper', 'body', 'html', '#document', 'Window'], 'userId': None, 'useraleVersion': 'demo', 'location': {'x': 462, 'y': 688}, 'toolName': None, 'toolVersion': None, 'clientTime': 1569123638986, 'details': {'alt': False, 'shift': False, 'meta': False, 'clicks': 1, 'ctrl': False}, '@timestamp': '2019-09-22T03:

In [6]:
#print query
print(LessAleS.to_dict())

{'query': {'match': {'type': 'click'}}}


In [65]:
#add query, filter to search params
FilterAleS = AleS \
    .query("match", type="click") \
    .filter("term", target="a.ui big blue button")

#print into pd df
AleList = []
for hit in FilterAleS.scan():
    AleList.append(hit.to_dict())
    
AleDf = json_normalize(AleList, sep="_")

AleDf

,@timestamp,clientTime,details_alt,details_clicks,details_ctrl,details_meta,details_shift,host,location_x,location_y,path,tags,target,toolName,toolVersion,type,userAction,userId,useraleVersion
0,2019-09-22T03:41:11.890Z,1569123669151,False,1,False,False,False,172.18.0.1,657,705,"[a.ui big blue button, div.ui center aligned t...",[_grokparsefailure],a.ui big blue button,None,None,click,True,None,demo
1,2019-09-22T03:41:13.360Z,1569123673249,False,1,False,False,False,172.18.0.1,602,694,"[a.ui big blue button, div.ui center aligned t...",[_grokparsefailure],a.ui big blue button,None,None,click,True,None,demo


In [66]:
#Or in Series-like object
AleDf.transpose()

,0,1
@timestamp,2019-09-22T03:41:11.890Z,2019-09-22T03:41:13.360Z
clientTime,1569123669151,1569123673249
details_alt,False,False
details_clicks,1,1
details_ctrl,False,False
details_meta,False,False
details_shift,False,False
host,172.18.0.1,172.18.0.1
location_x,657,602
location_y,705,694


In [69]:
AleDf.target

0    a.ui big blue button
1    a.ui big blue button
Name: target, dtype: object